In [1]:
## Simulationen laufen lassen und auswerten

In [2]:
# jeweils daten simulieren und vorauswerten; plots erstellen gehören in plots

# funktionen für:
# - parameter scan, beinhaltet Summe
# - 
# - Monte Carlo: Nein, zu viel Zeit, außer jemand hat besondere Langeweile

## Analysis File
This file is used to run and evaluate the simulations.<br>
For this are following libraries: <b>numpy</b> and <b>itertools</b> required.



In [3]:
import numpy as np
import pickle
from itertools import product
from functools import partial
import import_ipynb

In [4]:
from awareness import awareness_SIR
from simulation import simulate_rk4

importing Jupyter notebook from awareness.ipynb
importing Jupyter notebook from simulation.ipynb
importing Jupyter notebook from rk4.ipynb
importing Jupyter notebook from plots.ipynb


In [5]:
def axis_product(params: list, default_params: list)->tuple:
    """
    returnes combinations for parameters, varying one axis at a time
    params: list containing lists for different parameter values
    default_params: default values to use when another axis is being varied
    """
    combinations = []
    combinations.append(default_params)
    for i in range(len(default_params)):
        options = [params[j] if j == i else [default_params[j]] for j in range(len(default_params))]
        combinations += [x for x in product(*options)]
    combinations_filtered = []
    for comb in combinations:
        if comb not in combinations_filtered:
            combinations_filtered.append(comb)
    return combinations_filtered


In [6]:
def str_name(param_names: list, params: list)->str:
    s = '___'.join([f'{param_names[i]}_{params[i]:.2e}' for i in range(len(params))]) + '.pkl'
    return s


In [7]:
def save_data(data, fname):
    with open(fname, "wb+") as f:
        pickle.dump(data, f)

In [8]:
def load_data(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    t, y = data
    return t, y

In [9]:
def run_sensitivity_sweep(infection_params, awareness_params, awareness_params_variation, simulation_params):

    beta0, gamma0 = infection_params
    alpha0, omega0, lam0, rho0, kappa0 = awareness_params
    alpha, omega, lam, rho, kappa = awareness_params_variation
    grid_size, info_compartments, tmax, dt = simulation_params
    
    sweep_params = axis_product(awareness_params_variation, awareness_params)

    y0 = np.zeros(shape=(grid_size, grid_size, 3, info_compartments))  # start parameters
    y0[:,:,0,-1] = 1  # everyone is susceptible, no awareness at the start

    mid = int((grid_size-1)/2)
    y0[mid, mid, 1, -1] = 1  # one infection in the middle
    y0[mid, mid, 0, -1] = 0  # remove infected from susceptible pool

    model = partial(awareness_SIR, beta=beta0, gamma=gamma0)

    param_names = ['alpha', 'omega', 'lam', 'rho', 'kappa']

    # simulate different scenarios

    for i, param_set in enumerate(sweep_params):
        print(f'{i+1}/{len(sweep_params)}:   ', end='')
        params_dict = dict(zip(param_names, param_set))
        filename = 'data/' + str_name(param_names, param_set)
        simulate_rk4(model, y0, params_dict, tmax, dt, fname=filename)

    # analyze

    for i in range(len(awareness_params)):  # iterate over different parameters to vary
        options = [awareness_params_variation[j] if j == i else [awareness_params[j]] for j in range(len(awareness_params))]
        combinations = [x for x in product(*options)]

        param_variation = []
        concurrent_infections = []

        for param_set in combinations:
            params_dict = dict(zip(param_names, param_set))
            filename = 'data/' + str_name(param_names, param_set)
            t, y = load_data(filename)
            infections = np.average(np.sum(y[:, :, 1, :], axis=3), axis=(0, 1))
            max_infections = np.average(infections)

            param_variation.append(param_set[i])
            concurrent_infections.append(max_infections)
        
        data = (param_variation, concurrent_infections)
        save_data(data, f'data/variation_{param_names[i]}.pkl')



    print('done')



In [10]:
def run_information_simulation(infection_params, awareness_params, simulation_params):
    beta0, gamma0 = infection_params
    alpha0, omega0, lam0, rho0, kappa0 = awareness_params
    grid_size, info_compartments, tmax, dt = simulation_params
    
    y0 = np.zeros(shape=(grid_size, grid_size, 3, info_compartments))  # start parameters
    y0[:,:,0,-1] = 1  # everyone is susceptible, no awareness at the start

    mid = int((grid_size-1)/2)
    y0[mid, mid, 0, 0] = 1  # one aware dot in the middle
    y0[mid, mid, 0, -1] = 0  # remove unaware from susceptible pool

    model = partial(awareness_SIR, beta=beta0, gamma=gamma0)

    param_names = ['alpha', 'omega', 'lam', 'rho', 'kappa']

    # simulating

    params_dict = dict(zip(param_names, awareness_params))
    filename = 'data/information_only___' + str_name(param_names, awareness_params)
    simulate_rk4(model, y0, params_dict, tmax, dt, fname=filename)

    # analyze

    t, y = load_data(filename)
    data_av = np.average(y[:, :, :, :], axis=(0, 1)) # average over grid
    awareness_level = g2(rho0, data_av[0])  # everyone is susceptible, there are no infections
    save_data(((t, awareness_level), 'data/awareness.pkl'))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac0f8ce2-3132-47be-a4d1-6216636e93ff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>